<a href="https://colab.research.google.com/github/KengoTobita/keyword/blob/master/keyword01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# キーワード抽出

## 日本語用自然言語処理ライブラリ「GiNZA」

[GiNZA](https://megagonlabs.github.io/ginza/)とは、日本語の依存関係や構文解析ができる自然言語ライブラリである。<br>



In [1]:
!pip install "https://github.com/megagonlabs/ginza/releases/download/v1.0.2/ja_ginza_nopn-1.0.2.tgz"

    100% |████████████████████████████████| 122.4MB 242kB/s 
  Stored in directory: /root/.cache/pip/wheels/4e/ec/db/a4c22268bd4ca2e711284022ae0d55cc2244b9170a9033b438
Successfully built ja-ginza-nopn


In [2]:
import spacy
nlp = spacy.load('ja_ginza_nopn')
doc = nlp('依存構造解析の実験を行っています。')
for sent in doc.sents:
    for token in sent:
        print(token.i, token.orth_, token.lemma_, token.pos_, token.dep_, token.head.i)
    print('EOS')


0 依存 依存 NOUN compound 2
1 構造 構造 NOUN compound 2
2 解析 解析 NOUN nmod 4
3 の の ADP case 2
4 実験 実験 NOUN obj 6
5 を を ADP case 4
6 行っ 行う VERB root 6
7 て て SCONJ mark 6
8 い 居る AUX aux 6
9 ます ます AUX aux 6
10 。 。 PUNCT punct 6
EOS


## GiNZAを使用してニュース記事を分析
これからはGiNZAを使用して、ニュース記事から固有名詞を抽出する作業を行う。


1.   使用するニュース記事を用意
2.   GiNZAを使用して固有名詞のみを抽出
3. TF-IDFを使用して記事特有の単語を抽出する



### 今回使用する記事
分析する記事は以下のニュースの本文である。<br>
分析当時にyahooトップニュースになっているものをピックアップした。

1.   [「令和」最初の日　両陛下の車に集まった人々から歓声](https://news.yahoo.co.jp/pickup/6322151) <br>
           カテゴリー : ニュース
           文字数　: 1044文字
           
2.   [平成から令和へ。ゴールデンウィークの飲食店は好調？　知られざる大型連休における4つの苦労](https://news.yahoo.co.jp/byline/toryu/20190501-00124106/)<br>
           カテゴリー : 経済
           文字数　: 5261文字
3. [ジャニーズがオンラインストア開設 地方ファンからも歓喜の声](https://news.yahoo.co.jp/pickup/6322156)<br>
         カテゴリー : エンタメ
         文字数　: 713文字
4. [ソフトB武田　令和の危険球退場1号　2回まさか…ウィーラー頭部に死球](https://headlines.yahoo.co.jp/hl?a=20190501-00000178-spnannex-base)
         カテゴリー : スポーツニュース
         文字数　: 333文字
5. [フェイスブックがサイト刷新、恋愛機能などを追加](https://news.yahoo.co.jp/pickup/6322131)<br>
          カテゴリー : IT
          文字数　: 790文字

In [0]:
news_1 = """「令和」最初の日の1日、人々は新天皇陛下の即位を祝い、新時代の訪れに希望を新たにした。列島は広い範囲で曇りや雨だったが、各地でイベントや行事が開かれ、祝賀ムードが広がった。

　東京都千代田区の皇居・半蔵門前。午後3時近くに即位された新天皇陛下と新皇后雅子さまを乗せた車が通過すると、近くの沿道を埋め尽くした大勢の人々が「陛下！」「雅子さま！」「おめでとうございます！」と歓声を上げた。

　横浜市戸塚区の大学1年、牧野航大（こうだい）さん（19）は「天皇陛下が笑顔で手を振ってくださり、とても幸せな気持ちになった。お二人とも海外で生活された経験を生かし、いろいろな国と友好関係を築いていただければ」と興奮冷めやらぬ様子。

　東京都武蔵村山市の保育士、鳥居里美さん（34）は長女（11）とともに訪れた。「テレビでご即位のニュースを見ていたら、自分の目で見たくなってここまで来ました。きれいな雅子さまの姿を間近で見られて感激です」と語った。

　皇居正門（二重橋）付近には、外国人観光客の姿も。アルゼンチンから旅行に来たというミゲル・ファブリティさん（33）は「日本の天皇のことはよく分からないが、歴史的瞬間に立ち会えてうれしい。一生の思い出になる」と笑顔をみせた。

　渋谷区のJR渋谷駅前では、新しい時代の幕開けに、喜びや夢を語る若者の姿があった。静岡県から友人と訪れた大学4年、田中杏和（あんな）さん（21）は「みんなが喜んでいる雰囲気の中で時代が変わる瞬間に立ち会えてうれしい。個人的には就職も決まり、心機一転って感じです」と声を弾ませた。

　大阪府から観光で来た高校1年、飯盛皓紀（こうき）さん（15）は「将来は医者になり、海外で困っている人を助けたいので勉強を頑張る。時代が平成から令和に変わっても、僕がやらなきゃいけないことは変わらない」と未来を見据えた。

　大阪・道頓堀の大型ビジョンでは、陛下が「世界の平和を切に希望します」と初めておことばを述べる様子が生中継。横浜市青葉区から訪れた会社員、畑迫（はたさこ）朋子さん（44）は傘を差しながら足を止め、「陛下もおっしゃったように、戦争のない、平和な時代になってほしい」と願った。

　大阪市役所に設けられた記帳所には、市民らが並んだ。同市中央区の中学教員、中村明信さん（32）は「ご公務は大変だと思うが、お体に気をつけてほしい。平和で皆が笑顔あふれる時代に」と望んだ。【竹内麻子、川上珠実、千脇康平、柴山雄太、堀祐馬】"""

In [0]:
news_2 = """ゴールデンウィークの旅行客が過去最高
元号が平成から令和へとかわり、新しい時代が幕を開けました。

平成最後で令和最初のゴールデンウィークをどのようにして過ごしていますか。

2019年の4月27日（土）から5月6日（月）にも渡る10連休のゴールデンウィークが始まり、既に半ばとなりましたが、以下のように国内旅行も外国旅行も好調ということです。

JTBは4日、2019年のゴールデンウイーク（GW）の旅行動向見通しを発表した。国内と海外を合わせた旅行者数は前年比1.2%増の2467万人で、過去最高になる見通し。海外旅行は前年比6.9%増と伸びが大きく、平均支出も過去最高になる。新天皇即位に伴う10連休を受けて長期滞在向けのヨーロッパなどの人気が高い。

JTBがアンケートや販売状況、航空会社の予約状況から、4月25日～5月5日の旅行動向を予測した。海外旅行者数は66.2万人で前年比6.9%増、国内は2401万人と同1.1%増の見通し。国内と海外いずれも1969年の調査以来、過去最高と予測する。平日を挟んで休みが前半3日、後半4日と分かれた2018年より長期休暇が取りやすくなっているためだ。

出典:GW旅行者が過去最高、JTB 10連休で海外人気
国内と海外ともに旅行費用も増えており、旅行日数も多くなっているというのです。

　国内旅行人数は前年比1.1％増の2401万人と推計。平均費用は1.7％増の3万6800円を見込む。

　アンケートによると旅行日数は「1泊2日」（36.9％）が最多だが、「3泊4日」（16.1％）、「5泊6日」（3.5％）はそれぞれ前年より1.2ポイント増加した。

（中略）

　旅行先は「関東」（20.7％）が最多だが、前年比で1.0ポイント減。2位が「近畿」（14.3％）で0.9ポイント増。3位の「中国・四国」（12.9％）は、前年比では2.0ポイント増と最も高い。

　海外旅行は人数が6.9％増の66万2千人、平均費用で1.5％増の26万8千円と推計。旅行先は「東南アジア（タイ、シンガポールなど）」「ヨーロッパ」「ハワイ」がベスト3。

出典:GW10連休の旅行者数、国内・海外とも過去最高　ＪＴＢ見通し
前例のない10連休ということで、国内および国外への旅行が促進されるのは、理解できることでしょう。

飲食店は好調
では、飲食店の集客はどのような状況なのでしょうか。

旅行や移動が多いとされる大型連休では、飲食店の集客が鈍ると思われることがあります。

しかし実は、ゴールデンウィーク、お盆を含めた夏休み、年末年始といった長期休暇では、学校や会社や工場に依存していなければ、飲食店は賑わうというのが、飲食業界の常識です。特にイベントの開催場所付近であれば、非常に混雑することになります。

長期休暇に混雑する主な理由は、次の日も休みであれば外に出掛けやすくなったり、せっかくの長い休みなのでよいものを食べに行こうと考えたりするからです。

ただ、2019年のゴールデンウィークは10連休と前例のないほど長期に渡っており、旅行客も過去最高を記録しているだけに、これまでとは異なる様相を呈するかもしれません。

そこで、ホテルやレストランの関係者に尋ねたところ、10連休の間の飲食店は最後となる祝日の5月6日（月）を除いては満遍なく集客できており、例年以上というのです。またホテルの客室稼働率は非常に高くなっています。

なぜ今年のゴールデンウィークは非常に好調なのでしょうか。

前もって告知
最初に挙げる大きな理由は、10連休になることが前もって十分に告知されていたことです。

5月1日が祝日になると、なぜ10連休が実現するのか。理由は、国民の祝日について定める「祝日法」が、前後を祝日にはさまれた日は休日とすると規定しているためだ。

2019年のゴールデンウィークは元々、4月27～29日と5月3～6日がそれぞれ休みで、間を空けて3連休と4連休が続くことになっている。

5月1日が祝日となった場合、この日と4月29日（昭和の日）と5月3日（憲法記念日）にそれぞれはさまれる4月30日と5月2日も祝日法の規定で休日となるため、4月27日から5月6日の10連休が実現することになる。

出典:2019年のゴールデンウィークは10連休か。新天皇の5月1日即位に伴い、政府が検討
第一九七回

閣第一三号

　　　天皇の即位の日及び即位礼正殿の儀の行われる日を休日とする法律案

　天皇の即位の日及び即位礼正殿の儀の行われる日は、休日とする。

出典:天皇の即位の日及び即位礼正殿の儀の行われる日を休日とする法律案
早いところでは、一昨年2017年12月に放送されたANN NEWSで、2019年は10連休になりそうだと取り上げられていました。

3連休を超えるような大型連休であったとしても、メディアが宣伝していなければ、多くの人は日が近くなるまではあまり意識しないものでしょう。

それが、1年半以上も前から、前例のない10連休が訪れると報道されていれば、一生に一度の長期休暇なので遠いヨーロッパに旅行してみようだとか、海外旅行の経験のない方でも近くのアジアくらいなら訪れてみようだとか、海外は難しくてもせめて国内旅行に行ってみようだとか、考えるものです。

旅行を計画していない人でも、さすがに10連休は長いので、何もしないで自宅にいるのはもったいないと思っています。

2019年ゴールデンウィークに関する調査レポート　平成最後＆令和元年のゴールデンウィークは2人に1人が10連休を取得予定！
上記のぐるなびによるアンケートによれば＜31.9％が例年より外出機会が増加＞＜ゴールデンウィークのおでかけ予定は「グルメスポット」や「ショッピング」が上位に＞という結果になっているのです。

オフィス街や工場、学校の近くにある飲食店では、10連休の多くを休業にしていることも少なくありません。

東京都内の人の数は減っているかもしれませんが、外食しようとする人が多く、ゴールデンウィークを休業にする飲食店もあるために、オープンしている飲食店がより好調になっているのではないでしょうか。

令和のプロモーション
次に挙げられるのは、多くの飲食店、特にホテルのレストランでゴールデンウィーク期間を含む魅力的な令和プロモーションが行われていることです。

例えば、新元号が令和であると発表されたのは4月1日11時ですが、その日の16時にはグランド ハイアット 東京が、4月1日から6月30日にかけてステーキハウスとして名高い「オーク ドア」でマグナムボトルのワイン付いた「ゴールデン ジャイアント バーガー」を販売すると周知しました。

金粉をまとわせたバンズで黒毛和牛やフォアグラ、トリュフなどの高級食材を挟み、直径25cm、高さ15cm、総重量3kgにもなるラグジュアリーな巨大バーガーとなっており、6名から8名が食べられるボリュームとなっています。付属するマグナムボトルのワインは、シャンパーニュ、白ワイン、赤ワインから選べるのも特徴です。

同日の夜にはザ・リッツ・カールトン大阪が、令和をイメージしたカクテル「薫（かおる）～REIWA～」を4月1日から「ザ・バー」で販売すると発表しました。

翌4月2日には、ヨコハマ グランド インターコンチネンタル ホテルが平成最後のサンセットクルーズや令和を印字したチョコレートなど多彩な商品を、横浜ベイシェラトン ホテル＆タワーズは令和を記念した「祝 平成 ＆ 新元号記念プラン」をゴールデンウィーク期間限定に販売すると発表しています。

4月5日には、パレスホテル東京が、令和を迎えるにふさわしい「新たな時代への窓をひらく」をテーマにしたコラボレーションとして、4月22日から「パレスホテル東京×舘鼻則孝氏」を実施するとアナウンスしました。

これ以降も令和にちなんだフェアや商品がたくさん発表されています。

一休.com、OZmallといった大手のレストラン予約サイトでも、この超大型連休の売上を伸ばそうと、ゴールデンウィーク特集にとても力を入れているので、目にしたことがある人も多いことでしょう。

新元号が令和と決まるといなや、魅力的で興味をそそるプロモーションがすぐに発表されたことで、飲食店に訪れるモチベーションを高めることにつながったのではないでしょうか。

飲食店の苦労
大型連休で、飲食店の集客は好調であると紹介し、その理由を述べてきました。しかしその裏では、飲食店にとって大型連休ならではの苦労もあるのです。

飲食店の苦労や対策をみていきましょう。

スタッフの確保
集中した繁忙期が続くことから、スタッフの確保が重要となっています。最終日はそこまで忙しくないのでカウントしないとしても、混雑日は9日間も続くことになるのです。

キッチンスタッフやサービススタッフが少なくなると、料理の提供やレスポンスが遅くなったり、料理や接客の質が下がったりしてしまいます。

そうならないように、スタッフを確保しておかなければなりません。それにはシフトを早めに組んで調整したり、時給を上げて希望者を募りやすくしたりすることが必要です。それでも足りなければ、バックオフィスの社員が現場に出て手伝うことも考えなければなりません。

チェーン店であれば他店舗からのヘルプで乗り切るという方法もあるでしょう。チェーン店でなくとも、親しい個店では、互いの飲食店で手伝うこともあります。

スタッフの確保はオペレーションの最優先課題なので、飲食店は早い段階から努力しているのです。

新規客の増加
ゴールデンウィーク期間中には、普段よりも多くの新規客が訪れる可能性が高いでしょう。もちろん、飲食店にとっては新しい顧客を開拓することにつながるのでチャンスであることは間違いありません。

しかし、そこで飲食店の魅力が伝わらなければリピーターになってもらえないので、忙しいながらもしっかりとアピールする必要があります。

初めての客が多いということで、お勧めメニューやドリンク、コースを説明したり、トイレの場所や営業時間を訊かれたりすることも多いでしょう。その際にサービススタッフが適切に応対できなければなりません。

いつもよりたくさんの新規客が訪れることを見越して、サービススタッフに改めて教育しておくことが大切です。

関係業者の確認
飲食店は多くの関係業者と取引を行っており、食材や飲み物を購入して配送してもらったり、機材や什器のメンテナンスしてもらったりしています。

ゴールデンウィークでは通常とは異なる勤務体系になることも多いので、しっかりと確認しておき、飲食店の運営に支障がないように計画しておかなければなりません。

もしも機材や什器に不具合があった場合には、修理にきてもらえないこともあるので、事前にチェックしておくことが重要です。

銀行の休業
日本でもようやくキャッシュレスが促進されてきていますが、クレジットカード加盟店手数料の負担があったり、古い体質の現金主義であったりすることから、飲食業界ではまだ現金払いが中心となっており、小さい飲食店であればあるほどこの傾向が強いです。

したがって、お釣りをたくさん用意しておかなければなりませんが、銀行は休業しています。もちろん、ATMは稼働していますが、全ての日に稼働しているとは限らなかったり、ATMの現金が不足したりする可能性もあるでしょう。そのため、事前に普段よりも多額のお釣りを準備しておかなければなりません。

売上金を入金する場合には、土日祝日は手数料がかかるからといって、後でまとめて入金することもあるでしょう。しかし、10日間の売上金となると額が大きくなるだけに、店内に保管しておくことはセキュリティ的な観点から好ましくありません。手数料がかかっても普段通りに入金するべきです。

キャッシュレス化が進んでいない日本の飲食店では、ゴールデンウィーク期間中、現金の管理が大変となります。

飲食店への理解も大切
5月1日の新天皇の即位というハレの日に際して、10連休という超大型連休が設けられました。ハレの日にご馳走を食べることは、食文化を育んできた人として極めて自然な行為であり、親しい人たちと共に素晴らしい食体験を得られるよい機会です。

ただ、その裏では、いくら好況で嬉しい悲鳴とはいえ、休まずに営業している飲食店があり、大型連休ならではの苦労もあります。

飲食店に訪れる際には、飲食店の苦労や混雑を十分に理解しておき、心の余裕をもって大切な人との食事を楽しんでもらえたら幸いです。

ツイート
シェア
ブックマーク
"""

In [0]:
news_3 = """5月1日、ジャニーズ事務所は公式サイト「Johnny's net」でジャニーズショップのオンラインストアをオープンすると発表。ファンから喜びの声が殺到し、Twitterのトレンドワードランキングにランクインするなど話題となっている。

これまでジャニーズショップは原宿・名古屋・大阪・福岡の4店舗だけだった。公式サイトによると、「ジャニーズショップオンラインストア」のオープン日は5月6日。取扱いアイテムは生写真各種、フォトBook2018各種、ライブ写真集『JOHNNY'S LIVE COLLECTION 2018』だという。

さらにジャニーズJr.オンリーストア「Johnny's ISLAND STORE」の店舗が5月30日にオープンし、公式オンラインショップは今年7月のオープン予定だと明かした。

これに対して「ジャニショ」「ジャニショオンライン」「ジャニーズオンラインショップ」など関連ワードがTwitterでトレンド入り。大きな反響を呼んでいる。

ファンからは「わーい!!!!!!!　マジで嬉しいありがとう!!!!!　ジャニーズオンラインショップ最高」との声が続出。また「地方民にはありがたすぎる制度やね。ジャニーズオンラインショップ。最近行ってないからオープンしたら爆買いしそう。というか多分するよ」と地方在住のファンからも喜びの声が殺到した。

またオープン前に会員登録することができるため、「登録完了　地方住みの人にはいいかも　楽しみ!!!!」「ジャニーズオンラインショップ、事前登録完了！　いつでも買いに行けるわけではないので通販はとっても有難い!!!」とすでに登録を済ませたファンも多くみられた。"""

In [0]:
news_4 = """◇パ・リーグ　ソフトバンクー楽天（２０１９年５月１日　ヤフオクD）

　ソフトバンク・武田が1日の楽天戦に先発し、2回途中で危険投球を投じ退場となった。

【写真】武田の死球でウィーラーのヘルメットが吹っ飛ぶ

　2回無死一塁、楽天の5番・ウィーラーに投じたこの日26球目が頭部に直撃。敷田球審は即刻、危険球退場を宣告した。

　ウィーラーがグラウンドに倒れこむと、楽天ナインがベンチから飛び出し、一触即発のムードが漂った。

　改元初戦に先発した武田は3勝目を目指したマウンド。初回を1安打1三振で上々の立ち上がりだっただけに、チームは痛い降板となった。

　危険球退場は、4月21日の楽天戦でオリックスの松葉が2回に銀次への危険球で退場となって以来、今季4人目。"""

In [0]:
news_5 = """【AFP＝時事】他のユーザーに恋心を抱いているなら、フェイスブック（Facebook）が仲人役に──。交流サイト（SNS）最大手、米フェイスブックは先月30日、新たな恋愛機能を含むサイト刷新を発表した。

【写真11枚】発表会の様子

　フェイスブックの刷新は同社の開発者会議「F8」で発表されたもので、現実世界での人間関係を育て、少数の友人グループにとってより居心地の良い場を提供することが目的。23億人のユーザーを抱えるフェイスブックは、誤った情報の拡散やサイト悪用を防止できていないことへの批判を受け、サイトの焦点を小規模コミュニケーションへと移行する方針を発表しており、今回のデザイン刷新はこの方針に沿ったものだ。

　フェイスブックのアプリ部門責任者フィジー・シモ（Fidji Simo）氏はAFPに対し、新デザインではサイト体験の中心がグループに置かれると説明。新たに追加された恋愛、友達づくり、イベントの各機能は、人々が実世界で会うことを促すためのものだと語った。

　グループ参加を容易にするツールとしては、ユーザーが興味を持つかもしれないグループのおすすめ機能などが追加される。段階的な導入が予定されている新機能「Meet New Friends（新しい友達と出会う）」では、共通のコミュニティーで知り合いを増やしたいと思っている他のユーザーとつながることができる。また「イベント」タブを使えば、オンライン上の友達と実際に会う予定を立てやすくなる。

　フェイスブックはさらに、恋愛機能をフィリピン、シンガポール、ブラジル、チリなど14か国にも拡大すると発表。新機能「Secret Crush（秘密の恋心）」は、友達の中からで自分が気になっている人を選び、相手も同じく自分に興味を持っている場合にのみ通知が送られる仕組みとなる。【翻訳編集】 AFPBB News"""

かきは固有名詞を持ってくる関数

In [0]:
import spacy 
nlp = spacy.load("ja_ginza_nopn")

def news_ginza(doc):
  doc = nlp(doc)
  wordlist = []
  for sent in doc.sents:
    for token in sent:
      if token.pos_ == "PROPN": 　　　#PROPNは固有名詞 
        wordlist.append(token.lemma_) # lemmaは原型
  wordlist = list(set(wordlist)) 　　　　#単語の重複を処理
  return wordlist

In [37]:
news_ginza(news_1)

['田中',
 '半蔵門',
 '朋子',
 'ミゲル',
 '鳥居',
 '千脇',
 '牧野',
 '雅子',
 '柴山',
 '令和',
 'JR',
 '堀',
 'リティ',
 '道頓堀',
 '川上',
 '祐馬',
 'アルゼンチン',
 '横浜市戸塚区',
 '中村',
 '渋谷駅',
 '康平',
 '飯盛',
 '里美',
 '平成',
 '明信',
 '航大',
 '杏',
 '珠実',
 '雄太',
 '竹内',
 '日本',
 '大阪',
 '麻子',
 '皓紀']

In [36]:
news_ginza(news_2)

['関東',
 'ぐるなび',
 '則孝',
 'リッツ',
 'ハイアット',
 '中国',
 '舘鼻',
 '令和',
 '一休',
 'ハワイ',
 '横浜',
 'アジア',
 '薫',
 'ステーキハウス',
 '近畿',
 '四国',
 'シャンパーニュ',
 '東京',
 'かおる',
 'シンガポール',
 '平成',
 'カールトン',
 'タイ',
 'ann',
 'ヨコハマ',
 '日本',
 'シェラトン',
 'ヨーロッパ',
 '大阪']

In [38]:
news_ginza(news_3)

['ジャニーズ', 'John', 'book', 'ジャニ', '福岡', '名古屋', 'ツイッター', '原宿', '大阪']

In [39]:
news_ginza(news_5)

['ブラジル', 'フィジー', 'チリ', 'シモ', 'フェイスブック', 'AFP', 'Facebook', 'シンガポール', 'フィリピン']

In [0]:
a = """米Appleが4月30日（現地時間）に発表した第2四半期（1～3月）決算は、売上高は前年同期比5％減の580億1500ドル、純利益は16％減の115億6100万ドル（1株当たり2ドル46セント）だった。2四半期連続の減収となった。"""

In [44]:
news_ginza(a)

[]